#### Importing Libraries

In [1]:
from time import time
import cv2
import numpy as np
import glob
import os
from datetime import date, datetime
import pandas

### Feature Extraction

#### LBP Feature Extraction Class with 8 neighbour points and 1cm  radius
#### Total Bins for Histogram(of LBP) = 27

In [16]:
from skimage import feature
import numpy as np
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius
    def describe(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        return hist

#### initialising the LBP class Object with radius = 1 and neighbour points = 8

In [21]:
desc = LocalBinaryPatterns(8, 1)


#### LBPFeatureVector Function 
#### parameter : path of image 
##### working: - calculate  histogram of lbp values for each frame in a video clip and return avg  histogram values of all frames for the video

In [22]:
def findLBPFeatureVector(path):
    hist = []
    cnt=0
    vidObj = cv2.VideoCapture(path);
    while (vidObj.isOpened()):
        ret,image = vidObj.read()
        if(ret==False):
            break
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        curHist = desc.describe(image)
        hist.append(curHist)
        cnt+=1
    vidObj.release();
    return np.average(np.array(hist),axis = 0)

    

####  Iterate over each video clip (from 1-20) actors and store the lbp feature vector (histogram) in data and it's label in label 
#### audio-video folder contains videos of 1-20 actors


In [23]:
import glob
emotion = {1:'neutral',2:'calm',3:'happy',4:'sad',5:'angry',6:'fearful',7:'disgust',8:'surprise'}
print(emotion)
data = []
label = []
for i in range(1,9):
    files = glob.glob('audio-video\\**\\02-01-0{emotion}-01-01-01-*.mp4'.format(emotion = i))
    for file in files:
        print(file)
        hist = findLBPFeatureVector(file)
        data.append(hist)
        label.append(i)


{1: 'neutral', 2: 'calm', 3: 'happy', 4: 'sad', 5: 'angry', 6: 'fearful', 7: 'disgust', 8: 'surprise'}
audio-video\Actor_01\02-01-01-01-01-01-01.mp4
audio-video\Actor_02\02-01-01-01-01-01-02.mp4
audio-video\Actor_03\02-01-01-01-01-01-03.mp4
audio-video\Actor_04\02-01-01-01-01-01-04.mp4
audio-video\Actor_05\02-01-01-01-01-01-05.mp4
audio-video\Actor_06\02-01-01-01-01-01-06.mp4
audio-video\Actor_07\02-01-01-01-01-01-07.mp4
audio-video\Actor_08\02-01-01-01-01-01-08.mp4
audio-video\Actor_09\02-01-01-01-01-01-09.mp4
audio-video\Actor_10\02-01-01-01-01-01-10.mp4
audio-video\Actor_11\02-01-01-01-01-01-11.mp4
audio-video\Actor_12\02-01-01-01-01-01-12.mp4
audio-video\Actor_13\02-01-01-01-01-01-13.mp4
audio-video\Actor_14\02-01-01-01-01-01-14.mp4
audio-video\Actor_15\02-01-01-01-01-01-15.mp4
audio-video\Actor_16\02-01-01-01-01-01-16.mp4
audio-video\Actor_17\02-01-01-01-01-01-17.mp4
audio-video\Actor_18\02-01-01-01-01-01-18.mp4
audio-video\Actor_19\02-01-01-01-01-01-19.mp4
audio-video\Actor_20\02

In [25]:
data = np.array(data)
label = np.array(label)
print(data.shape)
print(label.shape)

(157, 10)
(157,)


#### Linear Support Vector Classifier used for training the model and predicting the emotion of test video

In [26]:
from sklearn.svm import LinearSVC
print(len(label))
# print(data)
model = LinearSVC(random_state=42)
model.fit(data, label)

   

157


LinearSVC(random_state=42)

#### Saving and loading the model using pickle

In [27]:
import pickle
with open('finalTrainedModelv2_pkl', 'wb') as files:
    pickle.dump(model, files)

In [28]:
with open('finalTrainedModelv2_pkl' , 'rb') as f:
    lr = pickle.load(f)

display(lr)

LinearSVC(random_state=42)

#### Testing the model on 21-24 actors

In [29]:
test_data=[]
test_true_label = []
for i in range(1,9):
    test_files = []
    test_files = glob.glob('test\\**\\02-01-0{emotion}-01-01-01-*.mp4'.format(emotion = i))
    for file in test_files:
        print(file)
        hist = findLBPFeatureVector(file)
        test_data.append(hist)
        test_true_label.append(i)

test\Actor_21\02-01-01-01-01-01-21.mp4
test\Actor_22\02-01-01-01-01-01-22.mp4
test\Actor_23\02-01-01-01-01-01-23.mp4
test\Actor_24\02-01-01-01-01-01-24.mp4
test\Actor_21\02-01-02-01-01-01-21.mp4
test\Actor_22\02-01-02-01-01-01-22.mp4
test\Actor_23\02-01-02-01-01-01-23.mp4
test\Actor_24\02-01-02-01-01-01-24.mp4
test\Actor_21\02-01-03-01-01-01-21.mp4
test\Actor_22\02-01-03-01-01-01-22.mp4
test\Actor_23\02-01-03-01-01-01-23.mp4
test\Actor_24\02-01-03-01-01-01-24.mp4
test\Actor_21\02-01-04-01-01-01-21.mp4
test\Actor_22\02-01-04-01-01-01-22.mp4
test\Actor_23\02-01-04-01-01-01-23.mp4
test\Actor_24\02-01-04-01-01-01-24.mp4
test\Actor_21\02-01-05-01-01-01-21.mp4
test\Actor_22\02-01-05-01-01-01-22.mp4
test\Actor_23\02-01-05-01-01-01-23.mp4
test\Actor_24\02-01-05-01-01-01-24.mp4
test\Actor_21\02-01-06-01-01-01-21.mp4
test\Actor_22\02-01-06-01-01-01-22.mp4
test\Actor_23\02-01-06-01-01-01-23.mp4
test\Actor_24\02-01-06-01-01-01-24.mp4
test\Actor_21\02-01-07-01-01-01-21.mp4
test\Actor_22\02-01-07-01

In [30]:
test_true_label = np.array(test_true_label)
test_predict_label = np.array(model.predict(test_data))
display(test_predict_label.shape)

(32,)

#### Printing the predicted and true labels  for emotions of video

In [31]:

print(test_predict_label)
print(test_true_label.shape)
print(test_true_label)

[2 2 1 3 2 2 1 3 2 2 1 3 2 2 1 3 2 2 5 3 2 2 1 3 2 2 5 3 2 2 1 3]
(32,)
[1 1 1 1 2 2 2 2 3 3 3 3 4 4 4 4 5 5 5 5 6 6 6 6 7 7 7 7 8 8 8 8]


#### Calculating the Classification Report having Precision,Recall,f1 score,accuracy 

In [33]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(test_true_label,test_predict_label))
print(confusion_matrix(test_true_label,test_predict_label))

              precision    recall  f1-score   support

           1       0.17      0.25      0.20         4
           2       0.12      0.50      0.20         4
           3       0.12      0.25      0.17         4
           4       0.00      0.00      0.00         4
           5       0.50      0.25      0.33         4
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00         4

    accuracy                           0.16        32
   macro avg       0.11      0.16      0.11        32
weighted avg       0.11      0.16      0.11        32

[[1 2 1 0 0 0 0 0]
 [1 2 1 0 0 0 0 0]
 [1 2 1 0 0 0 0 0]
 [1 2 1 0 0 0 0 0]
 [0 2 1 0 1 0 0 0]
 [1 2 1 0 0 0 0 0]
 [0 2 1 0 1 0 0 0]
 [1 2 1 0 0 0 0 0]]


C:\Users\vishal bansal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vishal bansal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vishal bansal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

### Analysis

1) F1 score is very less and for classes 4,6,7,8 it is giving us incorrect results.
2) The model is not trained Well due to which we are not getting more values on diagonals of confusion matrix.
So all these suggest that model is not trained well and classes are not classified correctly.


### Improvement
1) We can use more parameters to extract more features like we can extract orientation etc using HOG Features.
2) We can also use sound and use librosa library to analyse the frequency and pitch for different emotions and use them to extract more features.
3) We can also move to non-linear classifier by fitting non-linear curve to classify the videos.
4) To deal with multiple features we can reduce the dimension using chi square distance.
